# Neo4j

In [32]:
import py2neo
import pandas as pd

In [3]:
graph = py2neo.Graph("bolt://localhost:7687", user="neo4j", password="111")

In [33]:
merged_df = pd.read_csv('merged_df.csv')
merged_df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Year', 'State', 'Risk', 'value',
       'State_name', 'Pop17', 'Pop18', 'Pop21', 'Precip17', 'Precip18',
       'Precip21', 'Temp17', 'Temp18', 'Temp21'],
      dtype='object')

In [34]:
merged_df['Precip21'] = merged_df['Precip21'].fillna((merged_df['Precip21'].median()))
merged_df['Temp21'] = merged_df['Temp21'].fillna((merged_df['Temp21'].median()))
merged_df
merged_df.to_csv('merged_df.csv')

In [5]:
query = """
load csv with headers from "file:///merged_df.csv" as neo
with neo
where neo.Year="2021"
merge (s:State{id:neo.State})
merge (r:Risk{id:neo.Risk})
merge (s)-[applies_to:year_2021{score:toInteger(neo.value)}]->(r)"""

nodes= graph.run(query)

In [11]:
query = """
match(n) detach delete n"""

nodes= graph.run(query)

In [35]:
query = """
load csv with headers from "file:///merged_df.csv" as neo
with neo
where neo.Year="2021"
merge (s:State{id:neo.State,pop:neo.Pop21,precip:neo.Precip21,tavg:neo.Temp21})
merge (r:Risk{id:neo.Risk})
merge (s)-[applies_to:year_2021{score:toInteger(neo.value)}]->(r)"""
nodes= graph.run(query)